### Bibliotecas

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons, load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.kernel_approximation import PolynomialCountSketch

from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from scipy import special

### Configurações gerais

In [ ]:
# Paleta de cor
cmap = plt.cm.Set3

### Gerando dados aleatórios - Toy data

In [ ]:
## Não funciona bem com esse kernel - Teste
#X, y = make_moons(n_samples=500, noise=0.08)

iris = load_iris()
X = iris.data[:, 2:] # Pegando as duas primeiras características
y = iris.target

### Plotando as amostras como pontos em um gráfico bidimensional

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=cmap)
plt.xlabel('Característica 1')
plt.ylabel('Característica 2')
plt.title('Conjunto de Dados de Classificação')
plt.show()

### Pre-processamento

In [ ]:
# Escalando os dados
scaler = StandardScaler()
X = scaler.fit_transform(X)

### Kernel bns_odd_kernel - NumPy

In [ ]:
class BnSplineOddKernel(BaseEstimator, TransformerMixin):
    """
    Implementação do kernel Bn-Spline de ordem ímpar para uso com o Scikit-learn.
    """
    def __init__(self, n):
        """
        n é o grau do polinômio de spline.
        """
        self.n = n
        
    def fit(self, X, y=None):
        """
        Ajusta o modelo aos dados de treinamento.
        """
        self.X = X
        return self
        
    def transform(self, X):
        """
        Transforma os dados de entrada no espaço de características.
        """
        K = np.zeros((len(X), len(self.X)))
        for i, x in enumerate(X):
            for j, y in enumerate(self.X):
                K[i, j] = self._bns_odd_kernel(x, y, self.n)
        return K
    
    def _bns_odd_kernel(self, X, Y, n):
        """
        Calcula o kernel Bn-Spline de ordem ímpar entre os vetores X e Y.
        """
        s = 0
        for k in range(n + 1):
            s += ((-1)**k * special.comb(n + 1, k) * special.comb(2*n + 1 - 2*k, n) *
                  np.sum(np.minimum(X, Y)**(2*n + 1 - 2*k) * np.maximum(X, Y)**(2*k - n - 1)))
        return s

In [ ]:
# Define o pipeline de transformação e modelo de classificação
model = Pipeline([
    ('transformer', BnSplineOddKernel(n=3)),
    ('classifier', SVC(kernel='precomputed'))
])

X_train_transformed = model['transformer'].fit_transform(X)

In [ ]:
# Treinando o modelo
model['classifier'].fit(X_train_transformed, y)

### Visualizando fronteira de decisão

In [ ]:
# Define os limites do gráfico
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.02), np.arange(y_min, y_max, 0.02))

# Faz previsões em toda a região do gráfico
Z = model.predict(model['transformer'].transform(np.c_[xx.ravel(), yy.ravel()]))
Z = Z.reshape(xx.shape)

# Plota a região de decisão e os pontos de treinamento
plt.contourf(xx, yy, Z, cmap=plt.cm.coolwarm, alpha=0.5)
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=cmap)
plt.xlabel('Característica 1')
plt.ylabel('Característica 2')
plt.title(f'Fronteira de Decisão com Kernel Bn-Spline')
plt.show()